## Event detection ground truth vs video

In [ ]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font

from stressgait_analysis.dataset import StressGaitDataset
import numpy as np
%matplotlib widget
%load_ext autoreload
from scipy.signal import find_peaks
%autoreload 2

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])

In [ ]:
dataset = StressGaitDataset(base_path, coarse_condition=True, specify_bouts=True)


### HS computation from force plates

In [ ]:
def hs_forceplate(fy):
        contact = np.where(fy > 20, 1, 0)
        hs = []
        to = []

        for i in range(1,len(contact)):
            if contact[i] == 1 and contact[i - 1] == 0:
                hs.append(i)

        return np.array(fy.index[hs])

### HS computation from video

In [ ]:
def hs_video(keypoints):
    dist = keypoints.left_ankle_x - keypoints.left_hip_x
    hs_idx, _ = find_peaks(dist, width=15, prominence=50)
    hs_v = keypoints.iloc[hs_idx].index
    return np.array(hs_v)

## Comparison

In [ ]:
differences = []
diff_per_trial = {}

for trial in dataset:

    try:
        forceplate_data = trial.load_force_plate_data(video_framerate=False)
    except:
        print("No force plate data for trial {}".format(trial))
        continue
    fy = forceplate_data["GRF_y_1"]
    keypoints = trial.load_keypoint_trajectories()


    hs_f = hs_forceplate(fy)
    hs_v = hs_video(keypoints)

    # they will have different lengths --> for the FP HS, get the corresponding video HS
    closest_pairs = []
    for hs in np.array(hs_f):

        closest_hs_v = min(hs_v, key=lambda event_l: abs(event_l - hs))

        closest_pairs.append((hs, closest_hs_v))
    hs_events = np.array(closest_pairs)

    #store in list
    differences.append(hs_events)


In [ ]:
all_differences  = np.concatenate(differences, axis=0)

In [ ]:
r_pearson = np.corrcoef(all_differences[:, 0], all_differences[:, 1])[0, 1]
print("Pearson correlation:", r_pearson)

In [ ]:
diff = all_differences[:, 0] - all_differences[:, 1]  #in seconds
diff_fr = diff*50 #in frames
mae = np.mean(np.abs(diff))
rmse = np.sqrt(np.mean(diff**2))
me = (np.mean(diff))

print("MAE:", mae)
print("RMSE:", rmse)
print("ME:", me)

## Repeat but add a 1 frame offset

In [ ]:
differences = []
diff_per_trial = {}

for trial in dataset:

    try:
        forceplate_data = trial.load_force_plate_data(video_framerate=False)
    except:
        print("No force plate data for trial {}".format(trial))
        continue
    fy = forceplate_data["GRF_y_1"]
    keypoints = trial.load_keypoint_trajectories()


    hs_f = hs_forceplate(fy)
    hs_v = hs_video(keypoints)

    # they will have different lengths --> for the FP HS, get the corresponding video HS
    closest_pairs = []
    for hs in np.array(hs_f):

        closest_hs_v = min(hs_v, key=lambda event_l: abs(event_l - hs))
        closest_hs_v +=0.02
        closest_pairs.append((hs, closest_hs_v))
    hs_events = np.array(closest_pairs)

    #store in list
    differences.append(hs_events)

In [ ]:
all_differences  = np.concatenate(differences, axis=0)

In [ ]:
r_pearson = np.corrcoef(all_differences[:, 0], all_differences[:, 1])[0, 1]
print("Pearson correlation:", r_pearson)

In [ ]:
diff = all_differences[:, 0] - all_differences[:, 1]  #in seconds
diff_fr = diff*50 #in frames
mae = np.mean(np.abs(diff))
rmse = np.sqrt(np.mean(diff**2))
me = (np.mean(diff))

print("MAE:", mae)
print("RMSE:", rmse)
print("ME:", me)